In [1]:
import requests
import pandas as pd
import sqlite3
import os
import time


def retry_requests(url, headers, payloads):
    
    for i in range(3):
        try:
            return requests.get(url, headers=headers, params=payloads)
        except:
            print('發生錯誤，等待1分鐘後嘗試')
            time.sleep(60)
    
    return None

def get_daily_prices(date):
        
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'
    
    payloads = {
        'response': 'html',
        'date': date,
        'type': 'ALLBUT0999'
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }
    
    response = retry_requests(url, headers, payloads)
    
    try:
        df = pd.read_html(response.text)[-1]
    except:
        return None
    
    df.columns = df.columns.get_level_values(2)
    
    df.drop(['證券名稱', '漲跌(+/-)'], inplace=True, axis=1)
    
    df['日期'] = pd.to_datetime(date)
    
    df = df.set_index(['證券代號', '日期'])
    
    df = df.apply(pd.to_numeric, errors='coerce')
    
    df.drop(df[df['收盤價'].isnull()].index, inplace=True)
    
    return df

def save_daily_prices(new_df):
    
    connection = sqlite3.connect('data.db')
    
    try:
        df = pd.read_sql('select * from daily_prices', connection, parse_dates=['日期'], index_col=['證券代號', '日期'])
    except:
        df = pd.DataFrame()
    
    combined_df = df.append(new_df, sort=False) 
    
    final_df = combined_df.reset_index().drop_duplicates(subset=['證券代號', '日期'], keep='last').set_index(['證券代號', '日期']).sort_index()
    
    final_df.to_sql('daily_prices', connection, if_exists='replace')
    
    connection.close()
    
    excel_file = os.path.join('data',
                              'excel_files',
                              'daily_prices.xlsx')
    
    os.makedirs(os.path.dirname(excel_file), exist_ok=True)
    
    final_df.to_excel(excel_file)   

In [3]:
for date in pd.date_range('20200301', '20200305'):
    print(date.strftime('%Y%m%d'))

20200301
20200302
20200303
20200304
20200305


In [4]:
def update_daily_prices(start_date, end_date):
    
    main_df = pd.DataFrame()

    for date in pd.date_range(start_date, end_date):

        df = get_daily_prices(date.strftime('%Y%m%d'))

        if df is not None:
            main_df = main_df.append(df)
            print('{} 抓取完成'.format(date.strftime('%Y%m%d')))
        else:
            print('{} 找不到資料'.format(date.strftime('%Y%m%d')))

        time.sleep(15)
        
    save_daily_prices(main_df)
    
    return main_df

In [5]:
update_daily_prices('2020/3/1', '2020/3/5')

20200301 找不到資料
20200302 抓取完成
20200303 抓取完成
20200304 抓取完成
20200305 抓取完成


,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,日期,,,,,,,,,,,,,
0050,2020-03-02,27894641,12628,2437814508,87.50,88.25,86.85,87.35,1.30,87.30,272,87.35,100,0.00
0051,2020-03-02,44100,31,1462989,33.24,33.42,32.78,33.23,0.26,33.20,2,33.23,27,0.00
0052,2020-03-02,179121,76,11997207,66.25,67.50,66.10,66.85,1.05,66.85,1,66.90,2,0.00
0053,2020-03-02,9010,9,352073,38.85,39.21,38.85,38.98,0.46,38.98,1,39.01,3,0.00
0054,2020-03-02,1000,1,22660,22.66,22.66,22.66,22.66,0.39,22.50,4,22.59,3,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2020-03-05,2094634,1070,43540322,21.25,21.40,20.50,20.65,0.80,20.65,6,20.70,5,0.00
9945,2020-03-05,1557510,879,66240415,42.10,42.70,42.10,42.60,0.65,42.55,62,42.60,16,3.93
9946,2020-03-05,848036,377,17164874,20.20,20.30,20.15,20.25,0.25,20.25,45,20.30,4,4.12
